# Mp 1 0 2_Read Ntuples Fill Histos And Fit
Read n-tuples in distinct workers, fill histograms, merge them and fit.
We express parallelism with multiprocessing as it is done with multithreading
in mt102_readNtuplesFillHistosAndFit.




**Author:** Danilo Piparo  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Saturday, October 30, 2021 at 08:23 AM.</small></i>

No nuisance for batch execution

In [1]:
gROOT->SetBatch();

---------------------------------------
 Perform the operation sequentially

In [2]:
TChain inputChain("multiCore");
inputChain.Add("mp101_multiCore_*.root");
if (inputChain.GetNtrees() <= 0) {
   Printf(" No files in the TChain: did you run mp101_fillNtuples.C before?");
   return 1;
}
TH1F outHisto("outHisto", "Random Numbers", 128, -4, 4);
inputChain.Draw("r >> outHisto");
outHisto.Fit("gaus");

 FCN=128.133 FROM MIGRAD    STATUS=CONVERGED      55 CALLS          56 TOTAL
                     EDM=3.28984e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.98681e+05   1.36749e+02   6.18632e-01  -3.46133e-08
   2  Mean         3.28613e-05   2.23731e-04   1.24141e-06   1.13020e-01
   3  Sigma        9.99986e-01   1.58890e-04   4.42086e-07  -8.75338e-02


Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


---------------------------------------
 We now go MP!
 TProcessExecutor offers an interface to directly process trees and chains without
 the need for the user to go through the low level implementation of a
 map-reduce.

We adapt our parallelisation to the number of input files

In [3]:
const auto nFiles = inputChain.GetListOfFiles()->GetEntries();

This is the function invoked during the processing of the trees.

In [4]:
auto workItem = [](TTreeReader &reader) {
   TTreeReaderValue<Float_t> randomRV(reader, "r");
   auto partialHisto = new TH1F("outHistoMP", "Random Numbers", 128, -4, 4);
   while (reader.Next()) {
      partialHisto->Fill(*randomRV);
   }
   return partialHisto;
};

Create the pool of processes

In [5]:
ROOT::TTreeProcessorMP workers(nFiles);

Process the tchain

In [6]:
auto sumHistogram = workers.Process(inputChain, workItem, "multiCore");
sumHistogram->Fit("gaus", 0);

return 0;

 FCN=128.133 FROM MIGRAD    STATUS=CONVERGED      49 CALLS          50 TOTAL
                     EDM=3.30619e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.98681e+05   1.36749e+02   6.18632e-01  -3.79011e-08
   2  Mean         3.28613e-05   2.23731e-04   1.24141e-06   1.12968e-01
   3  Sigma        9.99986e-01   1.58890e-04   4.42086e-07  -9.65899e-02


Draw all canvases 

In [7]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()